##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows how to classify images of flowers. It creates an image classifier using a `keras.Sequential` model, and loads data using `preprocessing.image_dataset_from_directory`. You will gain practical experience with the following concepts:

* Efficiently loading a dataset off disk.
* Identifying overfitting and applying techniques to mitigate it, including data augmentation and Dropout.

This tutorial follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process

## Import TensorFlow and other libraries

**Map Google Drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Get Drive Contents**

In [ ]:
!ls '/content'

**Check Available GPU**

In [ ]:
!nvidia-smi

**Import Required Libraries**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Download and explore the dataset

This tutorial uses a dataset of about 3,700 photos of flowers. The dataset contains 5 sub-directories, one per class:

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
!unzip -o '/content/drive/My Drive/Flowers/flower_photos.zip'

In [ ]:
import pathlib
data = pathlib.Path('/content/flower_photos')

In [ ]:
!ls '/content/flower_photos'

After downloading, you should now have a copy of the dataset available. There are 3,670 total images:

In [ ]:
image_count = len(list(data.glob('*/*.jpg')))
print(image_count)

Here are some roses:

In [ ]:
roses = list(data.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

And some tulips:

In [ ]:
tulips = list(data.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

# Load using keras.preprocessing

Let's load these images off disk using the helpful [image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory) utility. This will take you from a directory of images on disk to a `tf.data.Dataset` in just a couple lines of code. If you like, you can also write your own data loading code from scratch by visiting the [load images](https://www.tensorflow.org/tutorials/load_data/images) tutorial.

## Create a dataset

Define some parameters for the loader:

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

It's good practice to use a validation split when developing your model. Let's use 80% of the images for training, and 20% for validation.

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

You can find the class names in the `class_names` attribute on these datasets. These correspond to the directory names in alphabetical order.

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
dic = {k: v for v, k in enumerate(class_names)}
dic
thisdict = dict(map(reversed, dic.items()))
thisdict

## Visualize the data

Here are the first 9 images from the training dataset.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

You will train a model using these datasets by passing them to `model.fit` in a moment. If you like, you can also manually iterate over the dataset and retrieve batches of images:

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

The `image_batch` is a tensor of the shape `(32, 180, 180, 3)`. This is a batch of 32 images of shape `180x180x3` (the last dimension refers to color channels RGB). The `label_batch` is a tensor of the shape `(32,)`, these are corresponding labels to the 32 images. 

You can call `.numpy()` on the `image_batch` and `labels_batch` tensors to convert them to a `numpy.ndarray`.


## Configure the dataset for performance

Let's make sure to use buffered prefetching so you can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data.

`Dataset.cache()` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

`Dataset.prefetch()` overlaps data preprocessing and model execution while training. 

Interested readers can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Create the model

The model consists of three convolution blocks with a max pool layer in each of them. There's a fully connected layer with 128 units on top of it that is activated by a `relu` activation function. This model has not been tuned for high accuracy, the goal of this tutorial is to show a standard approach. 

In [ ]:
num_classes = 5

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Compile the model

For this tutorial, choose the `optimizers.Adam` optimizer and `losses.SparseCategoricalCrossentropy` loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Model summary

View all the layers of the network using the model's `summary` method:

In [ ]:
model.summary()

## Train the model

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Visualize training results

Create plots of loss and accuracy on the training and validation sets.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

As you can see from the plots, training accuracy and validation accuracy are off by large margin and the model has achieved only around 60% accuracy on the validation set.

Let's look at what went wrong and try to increase the overall performance of the model.

## Overfitting

In the plots above, the training accuracy is increasing linearly over time, whereas validation accuracy stalls around 60% in the training process. Also, the difference in accuracy between training and validation accuracy is noticeable—a sign of [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

When there are a small number of training examples, the model sometimes learns from noises or unwanted details from training examples—to an extent that it negatively impacts the performance of the model on new examples. This phenomenon is known as overfitting. It means that the model will have a difficult time generalizing on a new dataset.

There are multiple ways to fight overfitting in the training process. In this tutorial, you'll use *data augmentation* and add *Dropout* to your model.

## Data augmentation

Overfitting generally occurs when there are a small number of training examples. [Data augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation) takes the approach of generating additional training data from your existing examples by augmenting them using random transformations that yield believable-looking images. This helps expose the model to more aspects of the data and generalize better.

You will implement data augmentation using the layers from `tf.keras.layers.experimental.preprocessing`. These can be included inside your model like other layers, and run on the GPU.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

Let's visualize what a few augmented examples look like by applying data augmentation to the same image several times:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

You will use data augmentation to train a model in a moment.

## Dropout

Another technique to reduce overfitting is to introduce [Dropout](https://developers.google.com/machine-learning/glossary#dropout_regularization) to the network, a form of *regularization*.

When you apply Dropout to a layer it randomly drops out (by setting the activation to zero) a number of output units from the layer during the training process. Dropout takes a fractional number as its input value, in the form such as 0.1, 0.2, 0.4, etc. This means dropping out 10%, 20% or 40% of the output units randomly from the applied layer.

Let's create a new neural network using `layers.Dropout`, then train it using augmented images.

In [ ]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Compile and train the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
# model.save('/content/drive/My Drive/Models/FlowerImageClassification.h5')

## Visualize training results

After applying data augmentation and Dropout, there is less overfitting than before, and training and validation accuracy are closer aligned. 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Predict on new data

Finally, let's use our model to classify an image that wasn't included in the training or validation sets.

Note: Data augmentation and Dropout layers are inactive at inference time.

In [ ]:
# Map Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# Load Model
model = tf.keras.models.load_model('/content/drive/My Drive/Models/FlowerImageClassification.h5')

In [ ]:
import shutil
shutil.rmtree('/content/drive/My Drive/Flowers/TestImage/')

In [ ]:
!unzip -o '/content/drive/My Drive/Flowers/TestImage.zip' -d  '/content/drive/My Drive/Flowers/'

Archive:  /content/drive/My Drive/Flowers/TestImage.zip
  inflating: /content/drive/My Drive/Flowers/TestImage/1.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/12.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/2.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/3.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/4.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/45.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/5.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/55.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/6.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/66.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/7.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/75.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/8.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/81.jpg  
  inflating: /content/drive/My Drive/Flowers/TestImage/9

In [ ]:
sorted(os.listdir('/content/drive/My Drive/Flowers/TestImage'))

['1.jpg',
 '12.jpg',
 '2.jpg',
 '3.jpg',
 '4.jpg',
 '45.jpg',
 '5.jpg',
 '55.jpg',
 '6.jpg',
 '66.jpg',
 '7.jpg',
 '75.jpg',
 '8.jpg',
 '81.jpg',
 '9.jpg',
 '91.jpg',
 'daisy',
 'dandelion',
 'roses',
 'sunflowers',
 'tulips']

**Getting Folder names (for classes) and File names (for identification) into lists**

In [ ]:
files = next(os.walk('/content/drive/My Drive/Flowers/TestImage'))[2]
dirs = next(os.walk('/content/drive/My Drive/Flowers/TestImage'))[1]

In [ ]:
class_names = dirs
# del class_names[0]
class_names = sorted(class_names)
class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

**Creating dictionary for class names**

In [ ]:
dic = {k: v for v, k in enumerate(class_names)}
dic
thisdict = dict(map(reversed, dic.items()))
thisdict

{0: 'daisy', 1: 'dandelion', 2: 'roses', 3: 'sunflowers', 4: 'tulips'}

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

**Predicting on random image from web**

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to sunflowers with a 94.91 percent confidence.


**Getting list of images to be predicted from files list in TestFolder**

In [ ]:
data = []
for file in files:
  data.append((file))

df = pd.DataFrame(data, columns=['id'])


In [ ]:
import pathlib
datatest = pathlib.Path('/content/drive/My Drive/Flowers/TestImage/3.jpg')

**Predicting random image in TestFolder**

In [ ]:
img = keras.preprocessing.image.load_img(
datatest, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to tulips with a 55.67 percent confidence.


**Predicting For Test Set**

Preparing Test Data

We will prepare the test data by adding the path and file extension to the original test_set. This will help us load the images directly from the csv file using the load_img() method that you will see in the following code blocks.

In [ ]:
import pandas as pd
test_set = df
test_imgs = ['/content/drive/My Drive/Flowers/TestImage/{}'.format(x) for x in list(test_set.id)]
test_set = pd.DataFrame( {'Images': test_imgs })

**Predicting classes**

Now it’s time to load the images one by one and predict and store the category of each image from the test_set. 

In [ ]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(img_height, img_width))
  img = image.img_to_array(img)
  test_img = img.reshape((1,img_height, img_width,3))
  img_class = model.predict(test_img)
  prediction = img_class[0]
  Y_pred.append(prediction)

The above code block loads each image from the test set preprocess it and feeds it to the classifier to predict. The predictions are stored in a list called y_pred. Notice that all the image sizes (256,256) are the same as we did for the training set, this is important and otherwise would result in an error.

Let’s take a look at the predictions:

In [ ]:
print(Y_pred)

[array([-1.0763897 , -0.4567835 ,  0.19479296, -0.6548311 ,  3.045599  ],
      dtype=float32), array([ 2.7034745, -2.6535063, -0.5004432, -2.0236628,  2.563024 ],
      dtype=float32), array([-0.45467123, -5.623192  ,  2.7979932 , -4.9381976 ,  4.276346  ],
      dtype=float32), array([-1.7671432, -7.127079 ,  5.3755646, -2.271722 ,  5.6045856],
      dtype=float32), array([ 6.933498 , -2.31416  , -0.4526568, -5.5772495, -0.5034981],
      dtype=float32), array([ 0.5889915, -3.0389924,  0.7965386, -1.7821339,  3.7031028],
      dtype=float32), array([-2.1457257 ,  2.965099  , -0.2656238 ,  5.163875  , -0.24154076],
      dtype=float32), array([-0.7392759,  0.9183769,  3.1981077, -1.5844781,  2.5314   ],
      dtype=float32), array([ 0.40225554, 11.796281  , -4.345572  , -7.184286  , -2.0663993 ],
      dtype=float32), array([-3.7084732 , -0.10878256, -0.81180537,  5.3006635 ,  1.7044564 ],
      dtype=float32), array([-7.0030346,  3.2241075, -1.8965626,  7.2071757,  4.6745796],
      

**Converting prediction probabilities to class**

In [ ]:
ypred = tf.argmax(Y_pred, axis=1)

**Converting to list of predicted classes**

In [ ]:
pred = tf.Variable(ypred).numpy().tolist()

In [ ]:
print(pred)

[4, 0, 4, 4, 0, 4, 3, 2, 1, 3, 3, 3, 1, 1, 2, 1]


In [ ]:
prediction_classes = [thisdict.get(item,item) for item in pred]

In [ ]:
print(prediction_classes)

['tulips', 'daisy', 'tulips', 'tulips', 'daisy', 'tulips', 'sunflowers', 'roses', 'dandelion', 'sunflowers', 'sunflowers', 'sunflowers', 'dandelion', 'dandelion', 'roses', 'dandelion']


**Looking up prediction class numbers in dictionary to get class names dataframe in order of list**

In [ ]:
flower = []
#breed = []
for i in prediction_classes:
  flower.append(i[0:]) # First character = class_name/Animal
  #breed.append(i[1:]) # Last 2 characters = breed/Breed

In [ ]:
predictions = {}
predictions['class_name'] = flower
#predictions['breed'] = breed

In [ ]:
prediction = pd.DataFrame(predictions)

In [ ]:
prediction

,class_name
0,tulips
1,daisy
2,tulips
3,tulips
4,daisy
5,tulips
6,sunflowers
7,roses
8,dandelion
9,sunflowers


In [ ]:
prediction.groupby('class_name').count()

""
class_name
daisy
dandelion
roses
sunflowers
tulips


**Joining file list dataframe with prediction dataframe to get linewise prediction for each image file**

In [ ]:
detail_result = df.join(prediction)
detail_result 

,id,class_name
0,1.jpg,tulips
1,12.jpg,daisy
2,2.jpg,tulips
3,3.jpg,tulips
4,4.jpg,daisy
5,45.jpg,tulips
6,5.jpg,sunflowers
7,55.jpg,roses
8,6.jpg,dandelion
9,66.jpg,sunflowers


In [ ]:
prediction['class_name'].value_counts()

sunflowers    4
dandelion     4
tulips        4
roses         2
daisy         2
Name: class_name, dtype: int64

In [ ]:
# dandelion     3
# tulips        3
# sunflowers    2
# roses         1
# daisy         1
# Name: class_name, dtype: int64